In [1]:
#!pip install tab2img

In [2]:
from sklearn.datasets import fetch_covtype
from tab2img.converter import Tab2Img
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# model = Tab2Img()
# images = model.fit_transform(train, target)

In [66]:
train_data = pd.read_csv("data/titanic/train.csv")
test_data  = pd.read_csv("data/titanic/test.csv")

#replace NaN with 0
train_data.fillna(0,inplace=True)
test_data.fillna(0,inplace=True)

# drop useless columns
train_data = train_data.drop(columns=['Name','Cabin','Embarked', 'Ticket'])
train_data['TFamily'] = train_data['SibSp'] + train_data['Parch']
train_data = train_data.drop(columns=['SibSp', 'Parch'])

test_data = test_data.drop(columns=['Name','Cabin','Embarked', 'Ticket'])
test_data['TFamily'] = test_data['SibSp'] + test_data['Parch']
test_data = test_data.drop(columns=['SibSp', 'Parch'])

In [67]:
X = train_data.drop(columns=['Survived'])
Y = train_data['Survived'].values

In [68]:
arr = []
for sex in X['Sex']:
    if sex == 'male':
        arr.append(1)
    else:
        arr.append(0)
X['Sex'] = arr

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
#make the x for train and valid
xtrain,xvalid,ytrain,yvalid = train_test_split(X,Y,train_size=0.80,random_state=2)

In [81]:
model = Tab2Img()
images_train = model.fit_transform(xtrain.to_numpy(), ytrain)
images_valid = model.transform(xvalid.to_numpy())

In [85]:
images_train

array([40., 31.,  0.], dtype=float32)